In [1]:
using Arpack
using LinearAlgebra
using CairoMakie
using LaTeXStrings
using Revise
using Interpolations
using Roots
using ColorSchemes
using Colors
using QuadGK
using Integrals
using StaticArrays
include("/Users/avadali/Desktop/Desktop - Avi's Mac/Summer2024/AHC_code/Spinors/rmg_K.jl")
include("/Users/avadali/Desktop/Desktop - Avi's Mac/Summer2024/AHC_code/Lattice_setup/lattice_BZ.jl")
include("/Users/avadali/Desktop/Desktop - Avi's Mac/Summer2024/AHC_code/3P_funcs/patch_methods.jl")
include("/Users/avadali/Desktop/Desktop - Avi's Mac/Summer2024/AHC_code/3P_funcs/rmg_3pV.jl")

rmg_alpha_exp (generic function with 1 method)

In [3]:
function vF_eigs_grid(alpha, delta, grid, vF, index)
    eigen_vals = Array{Float64}(undef, size(grid, 1))
    for j in 1:length(eigen_vals)
        x = grid[j, 1]
        y = grid[j, 2]
        eigen_vals[j] = vF_analytic_eigenvalues(alpha, delta, x, y, vF)[index]
    end
    return eigen_vals
end

vF_eigs_grid (generic function with 1 method)

In [19]:
# Create grid
qx = range(-10, 10, 10^2)
qy = range(-10, 10, 10^2)
grid = mesh_grid(qx, qy, Float64)

delta = 1
alpha = exp(im*2*pi/3)
# vF values to animate over
vF_center = -2 * real(alpha)
vF_list = range(vF_center - 0.2, vF_center + 0.2, 100)

Z_extrema = map(vF_list) do vF
    Z = reshape(vF_eigs_grid(alpha/2, delta, grid, vF/2, 2), (100, 100))
    extrema(Z)
end

# Extract overall z-limits
zmins, zmaxs = map(first, Z_extrema), map(last, Z_extrema)
zmin, zmax = minimum(zmins), maximum(zmaxs)
zabs = maximum(abs.([zmin, zmax]))

# Set up the figure and axis
fig = Figure()
ax = Axis3(fig[1, 1]; xlabel=L"q_x", ylabel=L"q_y", zlabel=L"\epsilon_1(q_x, q_y)", title="")
# xlims!(ax, (-0.1, 0.1))
# ylims!(ax, (-0.1, 0.1))
ax.limits[] = ((minimum(qx), maximum(qx)),
               (minimum(qy), maximum(qy)),
               (zmin, zmax))  # Fixed Z-range
ax.elevation[] = pi/11       # 45 degrees from qx axis (adjust as needed)
# ax.azimuth[] = pi/10     # 45 degrees from qx axis (adjust as needed)
ax.xticks[] = ([-10, 0, 10], [L"-10\kappa", "0", L"10 \kappa"])
ax.yticks[] = ([-10, 0, 10], [L"-10\kappa", "0", L"10 \kappa"])
ax.zticks[] = ([-1], [L"-|\Delta|"])

ax_super = Axis(fig[:, :], title = L"\epsilon_1(q_x, q_y) \text{ vs } (q_x, q_y) \ | \Delta = 1, \ \alpha = e^{i 2 \pi / 3}", 
titlegap = 50)
hidedecorations!(ax_super, label = false)
hidespines!(ax_super)
# Record animation
record(fig, "band_animation.mp4", vF_list) do vF
    empty!(ax)  # Clear previous frame

    # Evaluate Z on the grid
    Z = reshape(vF_eigs_grid(alpha/2, delta, grid, vF/2, 2), (100, 100))

    # Plot surface
    tr = surface!(ax, qx, qy, Z; colormap = :RdBu, colorrange = (-zabs, zabs))
    # zmin, zmax = extrema(Z)
    # ax.limits[] = ((minimum(qx), maximum(qx)),
    #                (minimum(qy), maximum(qy)),
    #                (zmin, zmax))

    # Update the title
    ax.title = "vF = $(round(vF, digits=3))"
end

"band_animation.mp4"